In [1]:
import numpy as np
import cv2  # OpenCVライブラリ

import matplotlib.pyplot as plt 
%matplotlib inline

import torch

import pickle

import torch.utils.data as data
from itertools import product as product

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Function

In [2]:
# import dataset
from utils.dataset import VOCDataset, DatasetTransform, make_datapath_list, Anno_xml2list, od_collate_fn

In [3]:
# set your VOCdevkit path!
vocpath = "../VOCdevkit/VOC2007"
DEVKIT_PATH = "../VOCdevkit/"
SET = "test"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(vocpath)

In [4]:
len(val_img_list)

4952

In [5]:
val_anno_list[0]

'../VOCdevkit/VOC2007/Annotations/000001.xml'

In [6]:
image_index = []
for l in val_img_list:
    image_index.append(l[-10:-4])

image_index[0]

'000001'

In [7]:
class_names = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123)  # (BGR)の色の平均値
input_size = 300  # 画像のinputサイズを300×300にする

## DatasetTransformを適応
transform = DatasetTransform(input_size, color_mean)
transform_anno = Anno_xml2list(class_names)

In [8]:
val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DatasetTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(class_names))

In [9]:
val_dataloader = data.DataLoader(
    val_dataset, batch_size=1, shuffle=False, collate_fn=od_collate_fn, num_workers=8)

# set up model

In [21]:
from utils.feature_piramid_network import FPNSSD as SSD

voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']

# SSD300の設定
ssd_cfg = {
    'num_classes': 21,  # 背景クラスを含めた合計クラス数
    'input_size': 300,  # 画像の入力サイズ
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  # 出力するDBoxのアスペクト比の種類
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ
    'steps': [8, 16, 32, 64, 100, 300],  # DBOXの大きさを決める
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBOXの大きさを決める
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOXの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

# SSDネットワークモデル
net = SSD(phase="inference", cfg=ssd_cfg).eval()

# SSDの学習済みの重みを設定
checkpoint = torch.load("./weights/ssd_fpn_300_200.pth")
net.load_state_dict(checkpoint["state_dict"])
#optimizer.load_state_dict(checkpoint['optimizer'])
epoch = checkpoint['epoch']
print('loaded the trained weights')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using:", device)

net = net.to(device)

loaded the trained weights
using: cuda:0


In [22]:
all_imgs = []
classes = {}
bbox_threshold = 0.05

# define detections
all_boxes = [[[] for _ in range(len(val_img_list))]
               for _ in range(21)]
empty_array = np.transpose(np.array([[],[],[],[],[]]), (1,0))

In [23]:
from utils.ssd_predict_show import SSDPredictShow
ssd = SSDPredictShow(eval_categories=voc_classes, net=net, device=device)

cuda:0


# infer images

In [49]:
all_boxes = ssd.ssd_inference(val_dataloader, all_boxes, data_confidence_level=bbox_threshold)

(1, 21, 200, 5)
iter: 0
sort boxes. detection was 0.024934053421020508 and post took 0.00022745132446289062 and allboxappend took 0.00725102424621582
(1, 21, 200, 5)
iter: 1
sort boxes. detection was 0.020936965942382812 and post took 6.651878356933594e-05 and allboxappend took 0.006394147872924805
(1, 21, 200, 5)
iter: 2
sort boxes. detection was 0.11022281646728516 and post took 9.822845458984375e-05 and allboxappend took 0.0065081119537353516
(1, 21, 200, 5)
iter: 3
sort boxes. detection was 0.05880475044250488 and post took 7.557868957519531e-05 and allboxappend took 0.006587982177734375
(1, 21, 200, 5)
iter: 4
sort boxes. detection was 0.040550947189331055 and post took 8.726119995117188e-05 and allboxappend took 0.0061893463134765625
(1, 21, 200, 5)
iter: 5
sort boxes. detection was 0.02890300750732422 and post took 8.130073547363281e-05 and allboxappend took 0.006529331207275391
(1, 21, 200, 5)
iter: 6
sort boxes. detection was 0.0156705379486084 and post took 8.177757263183594e

(1, 21, 200, 5)
iter: 56
sort boxes. detection was 0.039995431900024414 and post took 7.43865966796875e-05 and allboxappend took 0.0064465999603271484
(1, 21, 200, 5)
iter: 57
sort boxes. detection was 0.011825323104858398 and post took 7.557868957519531e-05 and allboxappend took 0.0064394474029541016
(1, 21, 200, 5)
iter: 58
sort boxes. detection was 0.019884824752807617 and post took 0.00028443336486816406 and allboxappend took 0.0060787200927734375
(1, 21, 200, 5)
iter: 59
sort boxes. detection was 0.17828941345214844 and post took 8.511543273925781e-05 and allboxappend took 0.006924152374267578
(1, 21, 200, 5)
iter: 60
sort boxes. detection was 0.013610601425170898 and post took 6.270408630371094e-05 and allboxappend took 0.006423473358154297
(1, 21, 200, 5)
iter: 61
sort boxes. detection was 0.013220071792602539 and post took 6.556510925292969e-05 and allboxappend took 0.006258487701416016
(1, 21, 200, 5)
iter: 62
sort boxes. detection was 0.04486346244812012 and post took 7.67707

(1, 21, 200, 5)
iter: 117
sort boxes. detection was 0.032938480377197266 and post took 7.915496826171875e-05 and allboxappend took 0.006499767303466797
(1, 21, 200, 5)
iter: 118
sort boxes. detection was 0.10728168487548828 and post took 9.775161743164062e-05 and allboxappend took 0.006660938262939453
(1, 21, 200, 5)
iter: 119
sort boxes. detection was 0.013872385025024414 and post took 0.0003032684326171875 and allboxappend took 0.006281614303588867
(1, 21, 200, 5)
iter: 120
sort boxes. detection was 0.03904891014099121 and post took 7.796287536621094e-05 and allboxappend took 0.006375789642333984
(1, 21, 200, 5)
iter: 121
sort boxes. detection was 0.022386550903320312 and post took 6.628036499023438e-05 and allboxappend took 0.006420612335205078
(1, 21, 200, 5)
iter: 122
sort boxes. detection was 0.03905320167541504 and post took 7.510185241699219e-05 and allboxappend took 0.006358146667480469
(1, 21, 200, 5)
iter: 123
sort boxes. detection was 0.017005443572998047 and post took 7.72

(1, 21, 200, 5)
iter: 172
sort boxes. detection was 0.051405906677246094 and post took 9.036064147949219e-05 and allboxappend took 0.006515979766845703
(1, 21, 200, 5)
iter: 173
sort boxes. detection was 0.019512414932250977 and post took 0.00036644935607910156 and allboxappend took 0.006133317947387695
(1, 21, 200, 5)
iter: 174
sort boxes. detection was 0.0364987850189209 and post took 9.226799011230469e-05 and allboxappend took 0.00632023811340332
(1, 21, 200, 5)
iter: 175
sort boxes. detection was 0.01339101791381836 and post took 0.00010848045349121094 and allboxappend took 0.006393909454345703
(1, 21, 200, 5)
iter: 176
sort boxes. detection was 0.011628389358520508 and post took 7.963180541992188e-05 and allboxappend took 0.0064580440521240234
(1, 21, 200, 5)
iter: 177
sort boxes. detection was 0.015233039855957031 and post took 8.869171142578125e-05 and allboxappend took 0.006392478942871094
(1, 21, 200, 5)
iter: 178
sort boxes. detection was 0.01656317710876465 and post took 8.1

(1, 21, 200, 5)
iter: 227
sort boxes. detection was 0.03703880310058594 and post took 0.00010609626770019531 and allboxappend took 0.0065305233001708984
(1, 21, 200, 5)
iter: 228
sort boxes. detection was 0.03739809989929199 and post took 0.00010251998901367188 and allboxappend took 0.0064601898193359375
(1, 21, 200, 5)
iter: 229
sort boxes. detection was 0.04900217056274414 and post took 0.00010776519775390625 and allboxappend took 0.006656646728515625
(1, 21, 200, 5)
iter: 230
sort boxes. detection was 0.012459516525268555 and post took 7.081031799316406e-05 and allboxappend took 0.006184101104736328
(1, 21, 200, 5)
iter: 231
sort boxes. detection was 0.01630091667175293 and post took 0.00034236907958984375 and allboxappend took 0.00611567497253418
(1, 21, 200, 5)
iter: 232
sort boxes. detection was 0.015429258346557617 and post took 0.0002608299255371094 and allboxappend took 0.0060825347900390625
(1, 21, 200, 5)
iter: 233
sort boxes. detection was 0.013149738311767578 and post took

...

(1, 21, 200, 5)
iter: 4718
sort boxes. detection was 0.21783924102783203 and post took 0.00010824203491210938 and allboxappend took 0.0066738128662109375
(1, 21, 200, 5)
iter: 4719
sort boxes. detection was 0.1005549430847168 and post took 0.0001068115234375 and allboxappend took 0.0065081119537353516
(1, 21, 200, 5)
iter: 4720
sort boxes. detection was 0.16282033920288086 and post took 0.0005965232849121094 and allboxappend took 0.006158113479614258
(1, 21, 200, 5)
iter: 4721
sort boxes. detection was 0.0676872730255127 and post took 0.00010633468627929688 and allboxappend took 0.00658869743347168
(1, 21, 200, 5)
iter: 4722
sort boxes. detection was 0.016173839569091797 and post took 0.00010776519775390625 and allboxappend took 0.0062749385833740234
(1, 21, 200, 5)
iter: 4723
sort boxes. detection was 0.13984298706054688 and post took 0.00011157989501953125 and allboxappend took 0.0066013336181640625
(1, 21, 200, 5)
iter: 4724
sort boxes. detection was 0.04895472526550293 and post too

(1, 21, 200, 5)
iter: 4772
sort boxes. detection was 0.3802332878112793 and post took 0.0004277229309082031 and allboxappend took 0.006399393081665039
(1, 21, 200, 5)
iter: 4773
sort boxes. detection was 0.17970681190490723 and post took 0.00010371208190917969 and allboxappend took 0.0065495967864990234
(1, 21, 200, 5)
iter: 4774
sort boxes. detection was 0.01608729362487793 and post took 0.00030875205993652344 and allboxappend took 0.006249904632568359
(1, 21, 200, 5)
iter: 4775
sort boxes. detection was 0.5233280658721924 and post took 0.00011014938354492188 and allboxappend took 0.006715297698974609
(1, 21, 200, 5)
iter: 4776
sort boxes. detection was 0.020580291748046875 and post took 0.00029158592224121094 and allboxappend took 0.006026506423950195
(1, 21, 200, 5)
iter: 4777
sort boxes. detection was 0.01755666732788086 and post took 7.891654968261719e-05 and allboxappend took 0.0064394474029541016
(1, 21, 200, 5)
iter: 4778
sort boxes. detection was 0.018125534057617188 and post 

(1, 21, 200, 5)
iter: 4826
sort boxes. detection was 0.3104982376098633 and post took 0.0005044937133789062 and allboxappend took 0.0060749053955078125
(1, 21, 200, 5)
iter: 4827
sort boxes. detection was 0.05601835250854492 and post took 9.655952453613281e-05 and allboxappend took 0.0069522857666015625
(1, 21, 200, 5)
iter: 4828
sort boxes. detection was 0.015624284744262695 and post took 8.869171142578125e-05 and allboxappend took 0.0064907073974609375
(1, 21, 200, 5)
iter: 4829
sort boxes. detection was 0.1136937141418457 and post took 9.036064147949219e-05 and allboxappend took 0.010317325592041016
(1, 21, 200, 5)
iter: 4830
sort boxes. detection was 0.06572389602661133 and post took 0.00010585784912109375 and allboxappend took 0.006649017333984375
(1, 21, 200, 5)
iter: 4831
sort boxes. detection was 0.06156039237976074 and post took 9.822845458984375e-05 and allboxappend took 0.006395101547241211
(1, 21, 200, 5)
iter: 4832
sort boxes. detection was 0.21469998359680176 and post too

(1, 21, 200, 5)
iter: 4883
sort boxes. detection was 0.10758543014526367 and post took 0.00010204315185546875 and allboxappend took 0.006696224212646484
(1, 21, 200, 5)
iter: 4884
sort boxes. detection was 0.3197450637817383 and post took 0.000110626220703125 and allboxappend took 0.00664520263671875
(1, 21, 200, 5)
iter: 4885
sort boxes. detection was 0.398517370223999 and post took 0.0001251697540283203 and allboxappend took 0.006909370422363281
(1, 21, 200, 5)
iter: 4886
sort boxes. detection was 0.16619491577148438 and post took 9.72747802734375e-05 and allboxappend took 0.006844282150268555
(1, 21, 200, 5)
iter: 4887
sort boxes. detection was 0.04728245735168457 and post took 0.00010371208190917969 and allboxappend took 0.006966352462768555
(1, 21, 200, 5)
iter: 4888
sort boxes. detection was 0.3158886432647705 and post took 0.0004520416259765625 and allboxappend took 0.010687112808227539
(1, 21, 200, 5)
iter: 4889
sort boxes. detection was 0.18092870712280273 and post took 0.0001

(1, 21, 200, 5)
iter: 4939
sort boxes. detection was 0.39217472076416016 and post took 0.00010204315185546875 and allboxappend took 0.006876707077026367
(1, 21, 200, 5)
iter: 4940
sort boxes. detection was 0.03235363960266113 and post took 0.00010538101196289062 and allboxappend took 0.00665593147277832
(1, 21, 200, 5)
iter: 4941
sort boxes. detection was 0.18259549140930176 and post took 8.606910705566406e-05 and allboxappend took 0.006668806076049805
(1, 21, 200, 5)
iter: 4942
sort boxes. detection was 0.22260165214538574 and post took 0.00011873245239257812 and allboxappend took 0.008643865585327148
(1, 21, 200, 5)
iter: 4943
sort boxes. detection was 0.14012598991394043 and post took 9.799003601074219e-05 and allboxappend took 0.0066106319427490234
(1, 21, 200, 5)
iter: 4944
sort boxes. detection was 0.21345996856689453 and post took 0.00011157989501953125 and allboxappend took 0.006570577621459961
(1, 21, 200, 5)
iter: 4945
sort boxes. detection was 0.014649391174316406 and post t

In [50]:
all_boxes[7][0:10]

[array([], shape=(0, 5), dtype=float64),
 array([], shape=(0, 5), dtype=float64),
 [[0.07815008, 147.93915, 144.83173, 176.39555, 180.84424],
  [0.06916909, 146.304, 104.497894, 174.63792, 141.7738]],
 [[0.99972755, 200.09566, 250.84566, 283.02274, 295.78625],
  [0.9994204, 231.50804, 248.22755, 314.94492, 292.91858],
  [0.9988286, 6.4114876, 218.04218, 50.304016, 258.27353],
  [0.9974324, 5.802043, 259.78018, 50.97944, 300.81274],
  [0.9878027, 133.46977, 252.70741, 199.80788, 286.18082],
  [0.9715811, 159.62756, 247.9864, 222.10513, 280.46228],
  [0.9387188, 116.621445, 262.09827, 162.4412, 290.1867],
  [0.9352266, 114.4977, 272.57162, 157.07147, 301.54312],
  [0.901275, 0.8749768, 245.89014, 46.589325, 286.93634],
  [0.882528, 110.125145, 215.63112, 151.75627, 242.94318],
  [0.8755542, 73.615265, 238.20418, 104.22993, 264.99176],
  [0.8422374, 83.44323, 261.6507, 127.85587, 289.93094],
  [0.8411715, 90.77975, 234.69556, 121.93282, 261.68582],
  [0.8168516, 89.636475, 269.93567, 134.

# eval accuracy

In [24]:
# eval function
def voc_eval(detpath,
             annopath,
             imagesetfile,
             classname,
             cachedir,
             ovthresh=0.5,
             use_07_metric=False):
  """
  rec, prec, ap = voc_eval(detpath,
                              annopath,
                              imagesetfile,
                              classname,
                              [ovthresh],
                              [use_07_metric])
  Top level function that does the PASCAL VOC evaluation.
  detpath: Path to detections
      detpath.format(classname) should produce the detection results file.
  annopath: Path to annotations
      annopath.format(imagename) should be the xml annotations file.
  imagesetfile: Text file containing the list of images, one image per line.
  classname: Category name (duh)
  cachedir: Directory for caching the annotations
  [ovthresh]: Overlap threshold (default = 0.5)
  [use_07_metric]: Whether to use VOC07's 11 point AP computation
      (default False)
  """
  # assumes detections are in detpath.format(classname)
  # assumes annotations are in annopath.format(imagename)
  # assumes imagesetfile is a text file with each line an image name
  # cachedir caches the annotations in a pickle file

  # first load gt
  if not os.path.isdir(cachedir):
    os.mkdir(cachedir)
  cachefile = os.path.join(cachedir, '%s_annots.pkl' % imagesetfile)
  # read list of images
  with open(imagesetfile, 'r') as f:
    lines = f.readlines()
  imagenames = [x.strip() for x in lines]

  if not os.path.isfile(cachefile):
    # load annotations
    recs = {}
    for i, imagename in enumerate(imagenames):
      recs[imagename] = parse_rec(annopath.format(imagename))
      if i % 100 == 0:
        print('Reading annotation for {:d}/{:d}'.format(
          i + 1, len(imagenames)))
    # save
    #print('Saving cached annotations to {:s}'.format(cachefile))
    #with open(cachefile, 'wb') as f:
    #  pickle.dump(recs, f)
  else:
    # load
    with open(cachefile, 'rb') as f:
      try:
        recs = pickle.load(f)
      except:
        recs = pickle.load(f, encoding='bytes')

  # extract gt objects for this class
  class_recs = {}
  npos = 0
  for imagename in imagenames:
    R = [obj for obj in recs[imagename] if obj['name'] == classname]
    bbox = np.array([x['bbox'] for x in R])
    difficult = np.array([x['difficult'] for x in R]).astype(np.bool)
    det = [False] * len(R)
    npos = npos + sum(~difficult)
    class_recs[imagename] = {'bbox': bbox,
                             'difficult': difficult,
                             'det': det}

  # read dets
  detfile = detpath.format(classname)
  with open(detfile, 'r') as f:
    lines = f.readlines()

  splitlines = [x.strip().split(' ') for x in lines]
  image_ids = [x[0] for x in splitlines]
  confidence = np.array([float(x[1]) for x in splitlines])
  BB = np.array([[float(z) for z in x[2:]] for x in splitlines])

  nd = len(image_ids)
  tp = np.zeros(nd)
  fp = np.zeros(nd)

  if BB.shape[0] > 0:
    # sort by confidence
    sorted_ind = np.argsort(-confidence)
#    sorted_scores = np.sort(-confidence)
    BB = BB[sorted_ind, :]
    image_ids = [image_ids[x] for x in sorted_ind]

    # go down dets and mark TPs and FPs
    for d in range(nd):
      id = image_ids[d][-10:-4]
      #print(id)
      # catch bad detections
      try:
          R = class_recs[id]
      except:
        print("det not found")
        continue
        
      bb = BB[d, :].astype(float)
      ovmax = -np.inf
      BBGT = R['bbox'].astype(float)

      if BBGT.size > 0:
        # compute overlaps
        # intersection
        ixmin = np.maximum(BBGT[:, 0], bb[0])
        iymin = np.maximum(BBGT[:, 1], bb[1])
        ixmax = np.minimum(BBGT[:, 2], bb[2])
        iymax = np.minimum(BBGT[:, 3], bb[3])
        iw = np.maximum(ixmax - ixmin + 1., 0.)
        ih = np.maximum(iymax - iymin + 1., 0.)
        inters = iw * ih

        # union
        uni = ((bb[2] - bb[0] + 1.) * (bb[3] - bb[1] + 1.) +
               (BBGT[:, 2] - BBGT[:, 0] + 1.) *
               (BBGT[:, 3] - BBGT[:, 1] + 1.) - inters)

        overlaps = inters / uni
        ovmax = np.max(overlaps)
        jmax = np.argmax(overlaps)

      if ovmax > ovthresh:
        if not R['difficult'][jmax]:
          if not R['det'][jmax]:
            tp[d] = 1.
            R['det'][jmax] = 1
          else:
            fp[d] = 1.
      else:
        fp[d] = 1.

  # compute precision recall
  fp = np.cumsum(fp)
  tp = np.cumsum(tp)
  rec = tp / float(npos)
  # avoid divide by zero in case the first detection matches a difficult
  # ground truth
  prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
  ap = voc_ap(rec, prec, use_07_metric)

  return rec, prec, ap

In [25]:
pascal_classes = np.asarray(["__background__", 'aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor'])

PASCAL_CLASSES = pascal_classes

# write out detections for evaluation

In [26]:
import os 
def get_voc_results_file_template(cls):
        # VOCdevkit/results/VOC2007/Main/<comp_id>_det_test_aeroplane.txt
        filename = 'det_' + "val" + '_'+cls+'.txt'
        filedir = os.path.join(DEVKIT_PATH, 'results', 'VOC2007', 'Main')
        if not os.path.exists(filedir):
            os.makedirs(filedir)
        path = os.path.join(filedir, filename)
        return path


def write_voc_results_file(pascal_classes, all_boxes, image_index):
        for cls_ind, cls in enumerate(pascal_classes):
            if cls == '__background__':
                continue
            print('Writing {} VOC results file'.format(cls))
            filename = get_voc_results_file_template(cls)
            with open(filename, 'wt') as f:
                for im_ind, index in enumerate(image_index):
                    dets = np.asarray(all_boxes[cls_ind][im_ind])
                    if dets == []:
                        continue
                    # the VOCdevkit expects 1-based indices
                    for k in range(dets.shape[0]):
                        #print(dets[k, 0])
                        f.write('{:s} {:.3f} {:.1f} {:.1f} {:.1f} {:.1f}\n'.
                                format(index, dets[k, 0],
                                       dets[k, 1] + 1, dets[k, 2] + 1,
                                       dets[k, 3] + 1, dets[k, 4] + 1))
import xml.etree.ElementTree as ET
def parse_rec(filename):
  """ Parse a PASCAL VOC xml file """
  tree = ET.parse(filename)
  objects = []
  for obj in tree.findall('object'):
    obj_struct = {}
    obj_struct['name'] = obj.find('name').text
    obj_struct['pose'] = obj.find('pose').text
    obj_struct['truncated'] = int(obj.find('truncated').text)
    obj_struct['difficult'] = int(obj.find('difficult').text)
    bbox = obj.find('bndbox')
    obj_struct['bbox'] = [int(bbox.find('xmin').text),
                          int(bbox.find('ymin').text),
                          int(bbox.find('xmax').text),
                          int(bbox.find('ymax').text)]
    objects.append(obj_struct)

  return objects
def voc_ap(rec, prec, use_07_metric=False):
  """ ap = voc_ap(rec, prec, [use_07_metric])
  Compute VOC AP given precision and recall.
  If use_07_metric is true, uses the
  VOC 07 11 point method (default:False).
  """
  if use_07_metric:
    # 11 point metric
    ap = 0.
    for t in np.arange(0., 1.1, 0.1):
      if np.sum(rec >= t) == 0:
        p = 0
      else:
        p = np.max(prec[rec >= t])
      ap = ap + p / 11.
  else:
    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.], rec, [1.]))
    mpre = np.concatenate(([0.], prec, [0.]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
      mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
  return ap

In [27]:
write_voc_results_file(pascal_classes, all_boxes, val_img_list)

Writing aeroplane VOC results file
Writing bicycle VOC results file
Writing bird VOC results file
Writing boat VOC results file
Writing bottle VOC results file
Writing bus VOC results file
Writing car VOC results file
Writing cat VOC results file


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


Writing chair VOC results file
Writing cow VOC results file
Writing diningtable VOC results file
Writing dog VOC results file
Writing horse VOC results file
Writing motorbike VOC results file
Writing person VOC results file
Writing pottedplant VOC results file
Writing sheep VOC results file
Writing sofa VOC results file
Writing train VOC results file
Writing tvmonitor VOC results file


# evaluation

In [28]:
def python_eval(output_dir='output'):
        annopath = os.path.join(
            DEVKIT_PATH,
            'VOC2007',
            'Annotations',
            '{:s}.xml')
        imagesetfile = os.path.join(
            DEVKIT_PATH,
            'VOC2007',
            'ImageSets',
            'Main',
            SET + '.txt')
        cachedir = os.path.join(DEVKIT_PATH, 'annotations_cache')
        aps = []
        # The PASCAL VOC metric changed in 2010.
        # VOC07 metric is quite old so don't use.
        use_07_metric = False
        print('VOC07 metric? ' + ('Yes' if use_07_metric else 'No'))
        if not os.path.isdir(output_dir):
            os.mkdir(output_dir)
        for i, cls in enumerate(PASCAL_CLASSES):
            if cls == '__background__':
                continue
            filename = get_voc_results_file_template(cls)
            rec, prec, ap = voc_eval(
                filename, annopath, imagesetfile, cls, cachedir, ovthresh=0.5,
                use_07_metric=use_07_metric)
            aps += [ap]
            print('AP for {} = {:.4f}'.format(cls, ap))
            with open(os.path.join(output_dir, cls + '_pr.pkl'), 'wb') as f:
                pickle.dump({'rec': rec, 'prec': prec, 'ap': ap}, f)
        print('Mean AP = {:.4f}'.format(np.mean(aps)))
        print('~~~~~~~~')
        print('Results:')
        for ap in aps:
            print('{:.3f}'.format(ap))
        print('{:.3f}'.format(np.mean(aps)))
        print('~~~~~~~~')
        print('')
        print('--------------------------------------------------------------')
        print('Results computed with the **unofficial** Python eval code.')
        print('Results should be very close to the official MATLAB eval code.')
        print('Recompute with `./tools/reval.py --matlab ...` for your paper.')
        print('-- Thanks, The Management')
        print('--------------------------------------------------------------')

In [29]:
# evaluate detections
python_eval()

VOC07 metric? No
Reading annotation for 1/4952
Reading annotation for 101/4952
Reading annotation for 201/4952
Reading annotation for 301/4952
Reading annotation for 401/4952
Reading annotation for 501/4952
Reading annotation for 601/4952
Reading annotation for 701/4952
Reading annotation for 801/4952
Reading annotation for 901/4952
Reading annotation for 1001/4952
Reading annotation for 1101/4952
Reading annotation for 1201/4952
Reading annotation for 1301/4952
Reading annotation for 1401/4952
Reading annotation for 1501/4952
Reading annotation for 1601/4952
Reading annotation for 1701/4952
Reading annotation for 1801/4952
Reading annotation for 1901/4952
Reading annotation for 2001/4952
Reading annotation for 2101/4952
Reading annotation for 2201/4952
Reading annotation for 2301/4952
Reading annotation for 2401/4952
Reading annotation for 2501/4952
Reading annotation for 2601/4952
Reading annotation for 2701/4952
Reading annotation for 2801/4952
Reading annotation for 2901/4952
Readi

Reading annotation for 1301/4952
Reading annotation for 1401/4952
Reading annotation for 1501/4952
Reading annotation for 1601/4952
Reading annotation for 1701/4952
Reading annotation for 1801/4952
Reading annotation for 1901/4952
Reading annotation for 2001/4952
Reading annotation for 2101/4952
Reading annotation for 2201/4952
Reading annotation for 2301/4952
Reading annotation for 2401/4952
Reading annotation for 2501/4952
Reading annotation for 2601/4952
Reading annotation for 2701/4952
Reading annotation for 2801/4952
Reading annotation for 2901/4952
Reading annotation for 3001/4952
Reading annotation for 3101/4952
Reading annotation for 3201/4952
Reading annotation for 3301/4952
Reading annotation for 3401/4952
Reading annotation for 3501/4952
Reading annotation for 3601/4952
Reading annotation for 3701/4952
Reading annotation for 3801/4952
Reading annotation for 3901/4952
Reading annotation for 4001/4952
Reading annotation for 4101/4952
Reading annotation for 4201/4952
Reading an

Reading annotation for 1801/4952
Reading annotation for 1901/4952
Reading annotation for 2001/4952
Reading annotation for 2101/4952
Reading annotation for 2201/4952
Reading annotation for 2301/4952
Reading annotation for 2401/4952
Reading annotation for 2501/4952
Reading annotation for 2601/4952
Reading annotation for 2701/4952
Reading annotation for 2801/4952
Reading annotation for 2901/4952
Reading annotation for 3001/4952
Reading annotation for 3101/4952
Reading annotation for 3201/4952
Reading annotation for 3301/4952
Reading annotation for 3401/4952
Reading annotation for 3501/4952
Reading annotation for 3601/4952
Reading annotation for 3701/4952
Reading annotation for 3801/4952
Reading annotation for 3901/4952
Reading annotation for 4001/4952
Reading annotation for 4101/4952
Reading annotation for 4201/4952
Reading annotation for 4301/4952
Reading annotation for 4401/4952
Reading annotation for 4501/4952
Reading annotation for 4601/4952
Reading annotation for 4701/4952
Reading an

Reading annotation for 3501/4952
Reading annotation for 3601/4952
Reading annotation for 3701/4952
Reading annotation for 3801/4952
Reading annotation for 3901/4952
Reading annotation for 4001/4952
Reading annotation for 4101/4952
Reading annotation for 4201/4952
Reading annotation for 4301/4952
Reading annotation for 4401/4952
Reading annotation for 4501/4952
Reading annotation for 4601/4952
Reading annotation for 4701/4952
Reading annotation for 4801/4952
Reading annotation for 4901/4952
AP for pottedplant = 0.0000
Reading annotation for 1/4952
Reading annotation for 101/4952
Reading annotation for 201/4952
Reading annotation for 301/4952
Reading annotation for 401/4952
Reading annotation for 501/4952
Reading annotation for 601/4952
Reading annotation for 701/4952
Reading annotation for 801/4952
Reading annotation for 901/4952
Reading annotation for 1001/4952
Reading annotation for 1101/4952
Reading annotation for 1201/4952
Reading annotation for 1301/4952
Reading annotation for 1401